
Welcome to Week 2's exploration of Port Communication in Python! This notebook will teach you how to build networked applications by understanding and working with ports. Whether you're building client-server applications, microservices, or debugging network issues, understanding port communication is essential.

## What You'll Learn
- Understanding ports and network communication
- Creating TCP servers and clients
- Building HTTP servers
- Handling multiple connections
- Best practices for network programming

## Prerequisites
- Basic Python knowledge (covered in Week 1)
- Understanding of functions and error handling
- Familiarity with object-oriented programming

Let's start by installing the required packages:

In [ ]:
# Check if required packages are installed
try:
    import socket
    import threading
    import time
    import psutil
    print("✓ All required packages are available!")
except ImportError as e:
    print(f"✗ Missing package: {str(e)}")
    print("\nIMPORTANT: To install packages, run this command in your terminal (not in the notebook):")
    print("uv pip install psutil")
    print("\nWhy not install in the notebook?")
    print("- Installing via notebook (!pip install) only affects the current kernel")
    print("- Using uv in the terminal installs to your project's virtual environment")
    print("- This ensures consistency across all notebooks and scripts")
    print("\nAfter installing, restart the kernel to use the new package.")


A port is like a numbered doorway into your computer:
- Each port number (0-65535) can be used for different services
- Common ports: 80 (HTTP), 443 (HTTPS), 22 (SSH), 3306 (MySQL)
- Multiple applications can listen on different ports
- Only one application can use a specific port at a time

### Common Port Ranges:
1. Well-known ports (0-1023)
   - Reserved for standard services
   - Require administrative privileges
   - Examples: HTTP (80), HTTPS (443), FTP (21)

2. Registered ports (1024-49151)
   - Used by applications
   - Don't require special privileges
   - Good for custom applications

3. Dynamic ports (49152-65535)
   - Used for temporary connections
   - Available for any purpose

Let's explore what ports are currently in use on your system:

In [ ]:
def list_active_connections():
    """List all active network connections and their ports"""
    connections = psutil.net_connections()
    
    print("Active Network Connections:")
    print("-" * 50)
    print(f"{'Local Address':<20} {'Local Port':<10} {'Status':<10}")
    print("-" * 50)
    
    for conn in connections:
        if conn.laddr:  # Only show connections with local addresses
            addr = conn.laddr.ip
            port = conn.laddr.port
            status = conn.status
            print(f"{addr:<20} {port:<10} {status:<10}")

try:
    list_active_connections()
except Exception as e:
    print(f"Error listing connections: {e}")
    print("Note: Some systems may require elevated privileges to list all connections.")


Let's create a basic TCP server that:
1. Listens on a specific port
2. Accepts connections from clients
3. Receives and sends messages
4. Handles multiple clients simultaneously

This example will help you understand:
- How servers listen for connections
- How clients connect to servers
- Basic socket programming concepts
- Error handling in network programming

In [ ]:
class EchoServer:
    def __init__(self, host='localhost', port=12345):
        self.host = host
        self.port = port
        self.server_socket = None
        self.running = False
        
    def start(self):
        """Start the server"""
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # Allow port reuse (helpful for development)
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        
        try:
            self.server_socket.bind((self.host, self.port))
            self.server_socket.listen(5)
            self.running = True
            
            print(f"Server listening on {self.host}:{self.port}")
            
            # Start accepting connections in a separate thread
            accept_thread = threading.Thread(target=self._accept_connections)
            accept_thread.start()
            
        except Exception as e:
            print(f"Error starting server: {e}")
            self.stop()
    
    def _accept_connections(self):
        """Handle incoming connections"""
        while self.running:
            try:
                client_socket, address = self.server_socket.accept()
                print(f"New connection from {address}")
                
                # Handle each client in a separate thread
                client_thread = threading.Thread(
                    target=self._handle_client,
                    args=(client_socket, address)
                )
                client_thread.start()
                
            except Exception as e:
                if self.running:  # Only show error if server is still meant to be running
                    print(f"Error accepting connection: {e}")
    
    def _handle_client(self, client_socket, address):
        """Handle individual client connections"""
        try:
            while self.running:
                # Receive data from client
                data = client_socket.recv(1024).decode('utf-8')
                if not data:
                    break
                    
                print(f"Received from {address}: {data}")
                
                # Echo back to client
                response = f"Server received: {data}"
                client_socket.send(response.encode('utf-8'))
                
        except Exception as e:
            print(f"Error handling client {address}: {e}")
            
        finally:
            client_socket.close()
            print(f"Connection closed with {address}")
    
    def stop(self):
        """Stop the server"""
        self.running = False
        if self.server_socket:
            self.server_socket.close()
            print("Server stopped")

# Create and start the server
server = EchoServer()
server.start()

print("Server is running... (Stop kernel to stop the server)")


Now let's create a client that can connect to our server. This will demonstrate:
- How to establish connections
- Send and receive data
- Handle connection errors
- Proper resource cleanup

In [ ]:
class EchoClient:
    def __init__(self, host='localhost', port=12345):
        self.host = host
        self.port = port
        self.socket = None
        
    def connect(self):
        """Connect to the server"""
        try:
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.socket.connect((self.host, self.port))
            print(f"Connected to server at {self.host}:{self.port}")
            return True
        except Exception as e:
            print(f"Error connecting to server: {e}")
            return False
    
    def send_message(self, message):
        """Send a message to the server and get the response"""
        if not self.socket:
            print("Not connected to server")
            return
        
        try:
            # Send message
            self.socket.send(message.encode('utf-8'))
            
            # Get response
            response = self.socket.recv(1024).decode('utf-8')
            return response
        except Exception as e:
            print(f"Error sending/receiving message: {e}")
            return None
    
    def close(self):
        """Close the connection"""
        if self.socket:
            self.socket.close()
            print("Connection closed")

# Create a client and test it
client = EchoClient()
if client.connect():
    # Send a test message
    response = client.send_message("Hello, Server!")
    print(f"Server response: {response}")
    
    # Send another message
    response = client.send_message("How are you?")
    print(f"Server response: {response}")
    
    # Close the connection
    client.close()


While the echo server demonstrates basic port communication, let's create something more practical: a simple HTTP server. This will help you understand:
- How web servers work at a basic level
- HTTP request/response cycle
- Handling different types of requests
- Serving files over HTTP

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import json

class SimpleHTTPHandler(BaseHTTPRequestHandler):
    def _send_response(self, message, status=200):
        """Helper method to send a response"""
        self.send_response(status)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        response = json.dumps({"message": message}).encode('utf-8')
        self.wfile.write(response)
    
    def do_GET(self):
        """Handle GET requests"""
        if self.path == '/':
            self._send_response("Welcome to the simple HTTP server!")
        elif self.path == '/time':
            current_time = time.strftime("%Y-%m-%d %H:%M:%S")
            self._send_response(f"Current time is: {current_time}")
        else:
            self._send_response("Not found", 404)
    
    def do_POST(self):
        """Handle POST requests"""
        if self.path == '/echo':
            # Get the length of the data
            content_length = int(self.headers['Content-Length'])
            # Read the data
            post_data = self.rfile.read(content_length)
            # Echo it back
            self._send_response(f"Received: {post_data.decode('utf-8')}")
        else:
            self._send_response("Not found", 404)

def run_http_server(port=8000):
    """Run the HTTP server"""
    server_address = ('', port)
    httpd = HTTPServer(server_address, SimpleHTTPHandler)
    print(f"Starting HTTP server on port {port}...")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nShutting down server...")
        httpd.server_close()

# Run the server in a separate thread so it doesn't block the notebook
server_thread = threading.Thread(target=run_http_server)
server_thread.daemon = True  # Thread will close when notebook closes
server_thread.start()

print("HTTP Server is running on port 8000")
print("Try these URLs in your browser:")
print("  - http://localhost:8000/")
print("  - http://localhost:8000/time")